<a href="https://colab.research.google.com/github/Rula-Islait/Research/blob/main/AttentionImplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
class ReverseAttention(layers.Layer):
    def __init__(self, num_heads=8, key_dim=64):
        super(ReverseAttention, self).__init__()
        self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)

    def call(self, inputs):
        attention_output = self.mha(inputs, inputs)
        return tf.sigmoid(1 - attention_output) + tf.math.tanh(attention_output)

In [ ]:
class DynamicAttention(layers.Layer):
    def __init__(self, channels, num_heads=8, key_dim=64):
        super(DynamicAttention, self).__init__()
        self.channels = channels
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.mha = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)
        self.batch_norm = layers.BatchNormalization()
        self.dropout = layers.Dropout(0.3)

    def call(self, inputs):
        attention_output = self.mha(inputs, inputs)
        attention_output = self.batch_norm(attention_output)
        attention_output = self.dropout(attention_output)
        return inputs * attention_output + inputs

In [ ]:
class SequentialAttention(layers.Layer):
    def __init__(self, channels, num_heads=8, key_dim=64):
        super(SequentialAttention, self).__init__()
        self.channels = channels
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.mha = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)

    def call(self, inputs):
        x = layers.Conv2D(self.channels, (3, 3), padding='same', activation='relu')(inputs)
        attention_output = self.mha(x, x)
        attention_map = layers.Conv2D(self.channels, (3, 3), padding='same', activation='sigmoid')(attention_output)
        return inputs + (inputs * attention_map)

In [ ]:
class DenseAttention(tf.keras.layers.Layer):
    def __init__(self, channels):
        super(DenseAttention, self).__init__()
        self.channels = channels
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=channels)
        self.pool = tf.keras.layers.GlobalAveragePooling2D()

    def build(self, input_shape):
        self.query_dim = input_shape[-1]

    def call(self, inputs):
        pooled_inputs = self.pool(inputs)
        pooled_inputs = tf.expand_dims(pooled_inputs, 1)
        attention_output = self.mha(pooled_inputs, pooled_inputs)
        return inputs * tf.expand_dims(attention_output, 1) + inputs

    def compute_output_shape(self, input_shape):
        return tf.TensorShape([input_shape[0], input_shape[1], input_shape[2], self.channels])